In [1]:
import sys
sys.path.insert(0,'../')
from cov_3X2 import *

#only for python3
import importlib
reload=importlib.reload

In [2]:
#setup parameters
lmax_cl=2000
lmin_cl=2
l0=np.arange(lmin_cl,lmax_cl)

lmin_cl_Bins=lmin_cl+10
lmax_cl_Bins=lmax_cl-10
Nl_bins=40
l_bins=np.int64(np.logspace(np.log10(lmin_cl_Bins),np.log10(lmax_cl_Bins),Nl_bins+1))
lb=np.sqrt(l_bins[1:]*l_bins[:-1])

# l=np.unique(np.int64(np.logspace(np.log10(lmin_cl),np.log10(lmax_cl),Nl_bins*20))) #if we want to use fewer ell

do_cov=True
bin_cl=True #True

SSV_cov=False
tidal_SSV_cov=False

bin_xi=True
# theta_bins=np.logspace(np.log10(1./60),1,20)

In [3]:
#Setup redshift bins
zmin=0.3
zmax=2

z=np.linspace(0,5,200)
pzs=lsst_pz_source(z=z)
x=z<zmax
x*=z>zmin
z=z[x]
pzs=pzs[x]

ns0=26#+np.inf #ns=inf means shape noise is zero
nbins=3  # Number of tomographic bins
z_sigma=0.01
zs_bins=source_tomo_bins(zp=z,p_zp=pzs,ns=ns0,nz_bins=nbins,
                         ztrue_func=ztrue_given_pz_Gaussian,zp_bias=np.zeros_like(z),
                        zp_sigma=z_sigma*np.ones_like(z))

/home/hungjinh/anaconda2/envs/py36/lib/python3.6/site-packages/astropy/units/quantity.py:968: RuntimeWarning: divide by zero encountered in true_divide
  return super(Quantity, self).__rtruediv__(other)


In [4]:
pk_params={'non_linear':1,'kmax':30,'kmin':3.e-4,'nk':5000,'scenario':'dmo'}

In [14]:
kappaS = cov_3X2(zs_bins=zs_bins,l=l0,do_cov=do_cov,bin_cl=bin_cl,l_bins=l_bins,zg_bins=None,
               SSV_cov=SSV_cov,tidal_SSV_cov=tidal_SSV_cov,do_xi=False,use_window=False,
                 do_sample_variance=True,power_spectra_kwargs={'pk_func':'bary_pk','pk_params':pk_params,'cosmo_params':cosmo_fid},
               bin_xi=bin_xi)#ns=np.inf)

In [15]:
pk_params['scenario']='dmo'
%time clSG=kappaS.cl_tomo(pk_params=pk_params)

/home/hungjinh/anaconda2/envs/py36/lib/python3.6/site-packages/astropy/units/quantity.py:968: RuntimeWarning: divide by zero encountered in true_divide
  return super(Quantity, self).__rtruediv__(other)


CPU times: user 15.2 s, sys: 3.91 s, total: 19.1 s
Wall time: 8.8 s


In [7]:
%time clS=clSG['stack'].compute()

CPU times: user 13.5 s, sys: 44.7 s, total: 58.3 s
Wall time: 2.96 s


In [10]:
clS['cov'].shape

(240, 240)

In [11]:
clS['cl'].shape

(240,)

In [13]:
cosmo_fid

{'As': 2.2e-09,
 'Om': 0.3075,
 'Omb': 0.0486,
 'Omd': 0.2589,
 'Omk': 0.0,
 'Oml': 0.6925,
 'h': 0.6774,
 'mnu': 0.06,
 'ns': 0.965,
 's8': 0.817,
 'tau': 0.06,
 'w': -1,
 'wa': 0}